In [117]:
def parsemaze(mazestr):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    maze = [[col for col in row] for row in mazestr.split('\n') if len(row) > 0]
    
    portals = {}
    
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] in alphabet:
                ni = i + 1
                if ni >= 0 and ni < len(maze):
                    if maze[ni][j] == '.':
                        pre = maze[i-1][j]
                        label = pre + maze[i][j]
                        if label in portals:
                            portals[label] += [ni, j]
                        else:
                            portals[label] = [ni, j]
                        continue
                ni = i - 1
                if ni >= 0 and ni < len(maze):
                    if maze[ni][j] == '.':
                        post = maze[i+1][j]
                        label = maze[i][j] + post
                        if label in portals:
                            portals[label] += [ni, j]
                        else:
                            portals[label] = [ni, j]
                        continue
                nj = j + 1
                if nj >= 0 and nj < len(maze[0]):
                    if maze[i][nj] == '.':
                        pre = maze[i][j-1]
                        label = pre + maze[i][j]
                        if label in portals:
                            portals[label] += [i, nj]
                        else:
                            portals[label] = [i, nj]
                        continue
                nj = j - 1
                if nj >= 0 and nj < len(maze[0]):
                    if maze[i][nj] == '.':
                        post = maze[i][j+1]
                        label = maze[i][j] + post
                        if label in portals:
                            portals[label] += [i, nj]
                        else:
                            portals[label] = [i, nj]
                        continue
                        
    portpairs = {}
    for port in portals:
        if len(portals[port]) > 2:
            src = (portals[port][0], portals[port][1])
            dst = (portals[port][2], portals[port][3])
            portpairs[src] = dst
            portpairs[dst] = src
    
    return maze, portals, portpairs

In [142]:
def manhattan(src, dst):
    distance = 0
    for s, d in zip(src, dst):
        distance += abs(s - d)
    return distance


def astar(src, dst, maze, moveset=[[0, 1], [0, -1], [1, 0], [-1, 0]], portals=[]):
    openlist = []
    visitlist = []
    parents = {}
    
    openlist.append([manhattan(src, dst), 0, src[0], src[1]])
    
    found = False
    while len(openlist) > 0:
        visiting = openlist[0]
        openlist = openlist[1:]
        visitlist.append((visiting[2], visiting[3]))
        
        if visiting[2] == dst[0] and visiting[3] == dst[1]:
            found = True
            break
        
        for move in moveset:
            ni = visiting[2] + move[0]
            nj = visiting[3] + move[1]
            
            if (ni, nj) in visitlist:
                continue
            
            if ni >= 0 and nj >= 0 and ni < len(maze) and nj < len(maze[0]):
                if maze[ni][nj] == '.':
                    g = visiting[1] + 1
                    f = g + manhattan((ni, nj), dst)/50
                    parents[(ni, nj)] = (visiting[2], visiting[3])
                    openlist.append([f, g, ni, nj])
                    
        if (visiting[2], visiting[3]) in portals:
#             print(visiting[2], visiting[3], 'in portals')
            ni, nj = portals[(visiting[2], visiting[3])]
            if not (ni, nj) in visitlist:
                g = visiting[1] + 1
                f = g + manhattan((ni, nj), dst)/50
                parents[(ni, nj)] = (visiting[2], visiting[3])
                openlist.append([f, g, ni, nj])
                
        openlist.sort()
    
    path = []
    if found:
        cur = dst
        while not (cur == src):
            path.append(cur)
            cur = parents[cur]
        path.reverse()
    return path

In [143]:
with open('calca_20_input.txt', 'r') as infile:
    inlines = [line for line in infile if len(line) > 0]
    
mazestr = ''.join(inlines)

m, ports, pairs = parsemaze(mazestr)
    
xsrc = ports['AA']
xdst = ports['ZZ']
src = (xsrc[0], xsrc[1])
dst = (xdst[0], xdst[1])
path = astar(src, dst, m, portals=pairs)
print(len(path))

556


In [135]:
mazestr = """\
         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       """

m, ports, pairs = parsemaze(mazestr)
for p in ports:
    print(p, ports[p])
    
for pr in pairs:
    print(pr,'->', pairs[pr])

for i, r in enumerate(m):
    print(' '.join(r))
    
path = astar((2, 9), (16, 13), m, portals=pairs)
print(len(path), path)

FG [12, 11, 15, 2]
DE [10, 6, 13, 2]
AA [2, 9]
BC [6, 9, 8, 2]
ZZ [16, 13]
(6, 9) -> (8, 2)
(13, 2) -> (10, 6)
(12, 11) -> (15, 2)
(15, 2) -> (12, 11)
(10, 6) -> (13, 2)
(8, 2) -> (6, 9)
                  A                      
                  A                      
    # # # # # # # . # # # # # # # # #    
    # # # # # # # . . . . . . . . . #    
    # # # # # # # . # # # # # # # . #    
    # # # # # # # . # # # # # # # . #    
    # # # # # # # . # # # # # # # . #    
    # # # # #     B         # # # . #    
B C . . . # #     C         # # # . #    
    # # . # #               # # # . #    
    # # . . . D E     F     # # # . #    
    # # # # #         G     # # # . #    
    # # # # # # # # # . # # # # # . #    
D E . . # # # # # # # . . . # # # . #    
    # . # # # # # # # # # . # # # . #    
F G . . # # # # # # # # # . . . . . #    
    # # # # # # # # # # # . # # # # #    
                          Z              
                          Z              
23 [(3, 9), (4,

In [136]:
mazestr = """\
                   A               
                   A               
  #################.#############  
  #.#...#...................#.#.#  
  #.#.#.###.###.###.#########.#.#  
  #.#.#.......#...#.....#.#.#...#  
  #.#########.###.#####.#.#.###.#  
  #.............#.#.....#.......#  
  ###.###########.###.#####.#.#.#  
  #.....#        A   C    #.#.#.#  
  #######        S   P    #####.#  
  #.#...#                 #......VT
  #.#.#.#                 #.#####  
  #...#.#               YN....#.#  
  #.###.#                 #####.#  
DI....#.#                 #.....#  
  #####.#                 #.###.#  
ZZ......#               QG....#..AS
  ###.###                 #######  
JO..#.#.#                 #.....#  
  #.#.#.#                 ###.#.#  
  #...#..DI             BU....#..LF
  #####.#                 #.#####  
YN......#               VT..#....QG
  #.###.#                 #.###.#  
  #.#...#                 #.....#  
  ###.###    J L     J    #.#.###  
  #.....#    O F     P    #.#...#  
  #.###.#####.#.#####.#####.###.#  
  #...#.#.#...#.....#.....#.#...#  
  #.#####.###.###.#.#.#########.#  
  #...#.#.....#...#.#.#.#.....#.#  
  #.###.#####.###.###.#.#.#######  
  #.#.........#...#.............#  
  #########.###.###.#############  
           B   J   C               
           U   P   P               """

m, ports, pairs = parsemaze(mazestr)
for p in ports:
    print(p, ports[p])
    
for pr in pairs:
    print(pr,'->', pairs[pr])

for i, r in enumerate(m):
    print(' '.join(r))
    
xsrc = ports['AA']
xdst = ports['ZZ']
src = (xsrc[0], xsrc[1])
dst = (xdst[0], xdst[1])
path = astar(src, dst, m, portals=pairs)
print(len(path), path)

BU [21, 26, 34, 11]
LF [21, 32, 28, 15]
ZZ [17, 2]
YN [13, 26, 23, 2]
AA [2, 19]
JO [19, 2, 28, 13]
DI [15, 2, 21, 8]
QG [17, 26, 23, 32]
AS [8, 17, 17, 32]
JP [28, 21, 34, 15]
CP [8, 21, 34, 19]
VT [11, 32, 23, 26]
(8, 21) -> (34, 19)
(11, 32) -> (23, 26)
(21, 26) -> (34, 11)
(21, 32) -> (28, 15)
(15, 2) -> (21, 8)
(8, 17) -> (17, 32)
(34, 11) -> (21, 26)
(17, 32) -> (8, 17)
(13, 26) -> (23, 2)
(19, 2) -> (28, 13)
(23, 2) -> (13, 26)
(28, 21) -> (34, 15)
(28, 13) -> (19, 2)
(28, 15) -> (21, 32)
(23, 32) -> (17, 26)
(21, 8) -> (15, 2)
(23, 26) -> (11, 32)
(17, 26) -> (23, 32)
(34, 19) -> (8, 21)
(34, 15) -> (28, 21)
                                      A                              
                                      A                              
    # # # # # # # # # # # # # # # # # . # # # # # # # # # # # # #    
    # . # . . . # . . . . . . . . . . . . . . . . . . . # . # . #    
    # . # . # . # # # . # # # . # # # . # # # # # # # # # . # . #    
    # . # . # . . . . . . 

In [164]:
def parsemaze2(mazestr):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    maze = [[col for col in row] for row in mazestr.split('\n') if len(row) > 0]
    
    portals = {}
    
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] in alphabet:
                ni = i + 1
                if ni >= 0 and ni < len(maze):
                    if maze[ni][j] == '.':
                        pre = maze[i-1][j]
                        label = pre + maze[i][j]
                        if label in portals:
                            portals[label] += [ni, j]
                        else:
                            portals[label] = [ni, j]
                        continue
                ni = i - 1
                if ni >= 0 and ni < len(maze):
                    if maze[ni][j] == '.':
                        post = maze[i+1][j]
                        label = maze[i][j] + post
                        if label in portals:
                            portals[label] += [ni, j]
                        else:
                            portals[label] = [ni, j]
                        continue
                nj = j + 1
                if nj >= 0 and nj < len(maze[0]):
                    if maze[i][nj] == '.':
                        pre = maze[i][j-1]
                        label = pre + maze[i][j]
                        if label in portals:
                            portals[label] += [i, nj]
                        else:
                            portals[label] = [i, nj]
                        continue
                nj = j - 1
                if nj >= 0 and nj < len(maze[0]):
                    if maze[i][nj] == '.':
                        post = maze[i][j+1]
                        label = maze[i][j] + post
                        if label in portals:
                            portals[label] += [i, nj]
                        else:
                            portals[label] = [i, nj]
                        continue
                        
    portpairs = {}
    outerports = {}
    innerports = {}
    
#     mh = len(maze)//2
#     mw = len(maze[0])//2
#     print('centre', mh, mw)
    for port in portals:
        if len(portals[port]) > 2:
            src = (portals[port][0], portals[port][1])
            dst = (portals[port][2], portals[port][3])
            
            if src[0] == 2 or src[0] == len(maze)-3:
                outerports[src] = dst
                innerports[dst] = src
            elif src[1] == 2 or src[1] == len(maze[0])-3:
                outerports[src] = dst
                innerports[dst] = src
            else:
                outerports[dst] = src
                innerports[src] = dst
            
#             dsrc = manhattan(src, (mh, mw))
#             ddst = manhattan(dst, (mh, mw))
            
#             if dsrc < ddst:
#                 innerports[src] = dst
#                 outerports[dst] = src
#             else:
#                 innerports[dst] = src
#                 outerports[src] = dst
    
    return maze, portals, innerports, outerports

In [ ]:
def build_mazegraph(maze, portals, inners, outers):
    

In [165]:
def bfs2(src, dst, maze, moveset=[[0, 1], [0, -1], [1, 0], [-1, 0]], inners={}, outers={}, portnames={}):
    openlist = []
    visitlist = []
    parents = {}
    
    openlist.append([0, src[0], src[1], src[2]])
    
    found = False
    while len(openlist) > 0:
        visiting = openlist[0]
        level = visiting[3]
#         if (visiting[1], visiting[2]) in portnames:
#             print(visiting, 'visiting port', portnames[(visiting[1], visiting[2])], 'level', level)
        openlist = openlist[1:]
        visitlist.append((visiting[1], visiting[2], visiting[3]))
        
        if visiting[1] == dst[0] and visiting[2] == dst[1] and visiting[3] == dst[2]:
            found = True
            break
        
        for move in moveset:
            ni = visiting[1] + move[0]
            nj = visiting[2] + move[1]
            
            if (ni, nj, level) in visitlist:
                continue
            
            if ni >= 0 and nj >= 0 and ni < len(maze) and nj < len(maze[0]):
                if maze[ni][nj] == '.':
                    g = visiting[0] + 1
                    parents[(ni, nj, level)] = (visiting[1], visiting[2], level)
                    openlist.append([g, ni, nj, level])
                    
        if (visiting[1], visiting[2]) in inners:
            ni, nj = inners[(visiting[1], visiting[2])]
            if not (ni, nj, level+1) in visitlist:
                g = visiting[0] + 1
                parents[(ni, nj, level+1)] = (visiting[1], visiting[2], level)
                openlist.append([g, ni, nj, level+1])
        elif (visiting[1], visiting[2]) in outers:
            ni, nj = outers[(visiting[1], visiting[2])]
            if not (ni, nj, level-1) in visitlist and level >= 1:
                g = visiting[0] + 1
                parents[(ni, nj, level-1)] = (visiting[1], visiting[2], level)
                openlist.append([g, ni, nj, level-1])
                
        openlist.sort()
    
    path = []
    if found:
        cur = dst
        while not (cur == src):
            path.append(cur)
            cur = parents[cur]
        path.reverse()
    return path

In [167]:
with open('calca_20_input.txt', 'r') as infile:
    inlines = [line for line in infile if len(line) > 0]
    
mazestr = ''.join(inlines)

m, ports, ips, ops = parsemaze2(mazestr)
for p in ports:
#     print(p, ports[p])
    portlist = ports[p]
    pns[(portlist[0], portlist[1])] = p
    if len(portlist) > 2:
        pns[(portlist[2], portlist[3])] = p
    
print('maze size', len(m)-3, len(m[0])-3)
print('going down')
for ip in ips:
    print(pns[ip], ip,'->', ips[ip])
    
print('going up')
for op in ops:
    print(pns[op], op,'->', ops[op])    

# for i, r in enumerate(m):
#     print(' '.join(r))
    
xsrc = ports['AA']
xdst = ports['ZZ']

src = (xsrc[0], xsrc[1], 0)
dst = (xdst[0], xdst[1], 0)
    
path = bfs2(src, dst, m, inners=ips, outers=ops, portnames=pns)
print(len(path))

maze size 116 118
going down
GC (32, 65) -> (116, 79)
YA (86, 67) -> (81, 2)
CY (65, 88) -> (2, 37)
PY (32, 79) -> (41, 118)
UC (32, 57) -> (116, 61)
KY (79, 32) -> (81, 118)
TL (39, 88) -> (2, 49)
GY (32, 39) -> (116, 37)
JV (32, 81) -> (63, 118)
YO (32, 73) -> (43, 2)
YF (86, 73) -> (75, 2)
UM (32, 45) -> (35, 2)
AW (86, 39) -> (53, 2)
PP (86, 79) -> (2, 79)
XZ (75, 88) -> (2, 77)
DM (67, 88) -> (2, 65)
CQ (86, 51) -> (57, 118)
XI (49, 32) -> (116, 69)
KK (86, 47) -> (45, 118)
PN (75, 32) -> (63, 2)
GL (86, 59) -> (59, 2)
HB (57, 32) -> (116, 43)
LM (37, 32) -> (116, 75)
WH (43, 88) -> (116, 49)
NZ (65, 32) -> (2, 55)
CV (55, 88) -> (2, 69)
AM (45, 32) -> (67, 118)
going up
KK (45, 118) -> (86, 47)
AM (67, 118) -> (45, 32)
YF (75, 2) -> (86, 73)
PY (41, 118) -> (32, 79)
LM (116, 75) -> (37, 32)
UC (116, 61) -> (32, 57)
XI (116, 69) -> (49, 32)
DM (2, 65) -> (67, 88)
PN (63, 2) -> (75, 32)
GY (116, 37) -> (32, 39)
GC (116, 79) -> (32, 65)
WH (116, 49) -> (43, 88)
GL (59, 2) -> (86, 59

KeyboardInterrupt: 

In [168]:
mazestr = """\
             Z L X W       C                 
             Z P Q B       K                 
  ###########.#.#.#.#######.###############  
  #...#.......#.#.......#.#.......#.#.#...#  
  ###.#.#.#.#.#.#.#.###.#.#.#######.#.#.###  
  #.#...#.#.#...#.#.#...#...#...#.#.......#  
  #.###.#######.###.###.#.###.###.#.#######  
  #...#.......#.#...#...#.............#...#  
  #.#########.#######.#.#######.#######.###  
  #...#.#    F       R I       Z    #.#.#.#  
  #.###.#    D       E C       H    #.#.#.#  
  #.#...#                           #...#.#  
  #.###.#                           #.###.#  
  #.#....OA                       WB..#.#..ZH
  #.###.#                           #.#.#.#  
CJ......#                           #.....#  
  #######                           #######  
  #.#....CK                         #......IC
  #.###.#                           #.###.#  
  #.....#                           #...#.#  
  ###.###                           #.#.#.#  
XF....#.#                         RF..#.#.#  
  #####.#                           #######  
  #......CJ                       NM..#...#  
  ###.#.#                           #.###.#  
RE....#.#                           #......RF
  ###.###        X   X       L      #.#.#.#  
  #.....#        F   Q       P      #.#.#.#  
  ###.###########.###.#######.#########.###  
  #.....#...#.....#.......#...#.....#.#...#  
  #####.#.###.#######.#######.###.###.#.#.#  
  #.......#.......#.#.#.#.#...#...#...#.#.#  
  #####.###.#####.#.#.#.#.###.###.#.###.###  
  #.......#.....#.#...#...............#...#  
  #############.#.#.###.###################  
               A O F   N                     
               A A D   M                     """

m, ports, ips, ops = parsemaze2(mazestr)
for p in ports:
#     print(p, ports[p])
    portlist = ports[p]
    pns[(portlist[0], portlist[1])] = p
    if len(portlist) > 2:
        pns[(portlist[2], portlist[3])] = p
    
print('maze size', len(m), len(m[0]))
print('going down')
for ip in ips:
    print(pns[ip], ip,'->', ips[ip])
    
print('going up')
for op in ops:
    print(pns[op], op,'->', ops[op])    

# for i, r in enumerate(m):
#     print(' '.join(r))
    
xsrc = ports['AA']
xdst = ports['ZZ']

src = (xsrc[0], xsrc[1], 0)
dst = (xdst[0], xdst[1], 0)
    
path = bfs2(src, dst, m, inners=ips, outers=ops, portnames=pns)
print(len(path))

maze size 37 45
going down
NM (23, 36) -> (34, 23)
CK (17, 8) -> (2, 27)
FD (8, 13) -> (34, 19)
IC (8, 23) -> (17, 42)
LP (28, 29) -> (2, 15)
CJ (23, 8) -> (15, 2)
WB (13, 36) -> (2, 19)
RE (8, 21) -> (25, 2)
ZH (8, 31) -> (13, 42)
XQ (28, 21) -> (2, 17)
RF (21, 36) -> (25, 42)
OA (13, 8) -> (34, 17)
XF (28, 17) -> (21, 2)
going up
RF (25, 42) -> (21, 36)
FD (34, 19) -> (8, 13)
CJ (15, 2) -> (23, 8)
IC (17, 42) -> (8, 23)
ZH (13, 42) -> (8, 31)
XF (21, 2) -> (28, 17)
NM (34, 23) -> (23, 36)
LP (2, 15) -> (28, 29)
XQ (2, 17) -> (28, 21)
RE (25, 2) -> (8, 21)
WB (2, 19) -> (13, 36)
CK (2, 27) -> (17, 8)
OA (34, 17) -> (13, 8)
396


In [115]:
mazestr = """\
         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       """

m, ports, ips, ops = parsemaze2(mazestr)
pns = {}
for p in ports:
    print(p, ports[p])
    portlist = ports[p]
    pns[(portlist[0], portlist[1])] = p
    if len(portlist) > 2:
        pns[(portlist[2], portlist[3])] = p
    
print('going down')
for ip in ips:
    print(ip,'->', ips[ip])
    
print('going up')
for op in ops:
    print(op,'->', ops[op])    

for i, r in enumerate(m):
    print(' '.join(r))
    
xsrc = ports['AA']
xdst = ports['ZZ']

src = (xsrc[0], xsrc[1], 0)
dst = (xdst[0], xdst[1], 0)
    
path = bfs2(src, dst, m, inners=ips, outers=ops, portnames=pns)
print(len(path), path)

port FG (12, 11) is inner
port DE (10, 6) is inner
port BC (6, 9) is inner
FG [12, 11, 15, 2]
DE [10, 6, 13, 2]
AA [2, 9]
BC [6, 9, 8, 2]
ZZ [16, 13]
going down
(6, 9) -> (8, 2)
(10, 6) -> (13, 2)
(12, 11) -> (15, 2)
going up
(13, 2) -> (10, 6)
(8, 2) -> (6, 9)
(15, 2) -> (12, 11)
                  A                      
                  A                      
    # # # # # # # . # # # # # # # # #    
    # # # # # # # . . . . . . . . . #    
    # # # # # # # . # # # # # # # . #    
    # # # # # # # . # # # # # # # . #    
    # # # # # # # . # # # # # # # . #    
    # # # # #     B         # # # . #    
B C . . . # #     C         # # # . #    
    # # . # #               # # # . #    
    # # . . . D E     F     # # # . #    
    # # # # #         G     # # # . #    
    # # # # # # # # # . # # # # # . #    
D E . . # # # # # # # . . . # # # . #    
    # . # # # # # # # # # . # # # . #    
F G . . # # # # # # # # # . . . . . #    
    # # # # # # # # # # # . # # # # #    
    